In [ ]:
#habenula
rm(list=ls())    #clear workspace

library(devtools)
library(dplyr)
library(useful)
library(ggplot2)
#install_github("satijalab/seurat")
library(Seurat)
library(stringr)

#load habenula data
setwd('/Volumes/neurobio/MICROSCOPE/Daniel/inDrops/data/habenula')
data = read.table('habData.csv', header = T, fill = T, row.names = 1, sep =',')

#fill out

###########################################
#load in to seurat

hab = new('seurat', raw.data = data)

#500 min genes for cortical data, 300 min genes for hab
hab = Setup(hab, min.cells = 3, min.genes  = 300, do.logNormalize = T, total.expr = 10000, project = 'hab')

# We calculate the percentage of mitochondrial genes here and store it in percent.mito using the AddMetaData.
#The % of UMI mapping to MT-genes is a common scRNA-seq QC metric.

mito.genes = grep("^mt.", rownames(hab@data), value = T)
percent.mito = colSums(expm1(hab@data[mito.genes, ]))/colSums(expm1(hab@data))

#AddMetaData adds columns to object@data.info, and is a great place to stash QC stats
hab = AddMetaData(hab, percent.mito, "percent.mito")

VlnPlot(hab, c("nGene", "nUMI", "percent.mito"), nCol = 3)

#GenePlot is typically used to visualize gene-gene relationships, but can be used for anything calculated by the object,
#i.e. columns in object@data.info, PC scores etc.
par(mfrow = c(1, 2))
GenePlot(hab, "nUMI", "percent.mito")
GenePlot(hab, "nUMI", "nGene")

# cutoff cells with 50% of genes coming from mitochondrial transcripts.
hab = SubsetData(hab, subset.name = "percent.mito", accept.high = 0.5)

#how many cells did we just cutout?
length(hab@data[1,])

#how many cells are out from right v left?
length(grep('R', colnames(hab@data)))
length(grep('L', colnames(hab@data)))
#700 more from R

#RegressOut regresses out unimportant effects of gene variability...Look at raw code
hab = RegressOut(hab,latent.vars = c('nUMI','percent.mito'))

#detect variable geens
# I try to get roughly ~400 genes
hab = MeanVarPlot(hab ,fxn.x = expMean, fxn.y = logVarDivMean, x.low.cutoff = 0.0125, x.high.cutoff = 3, y.cutoff = 2.25, do.contour = F, num.bin = 100, do.text = F)

hab = PCA(hab, pc.genes = hab@var.genes, do.print = TRUE, pcs.print = 5, genes.print = 5)

#ProjectPCA scores each gene in the dataset (including genes not included in the PCA) based on their correlation with the calculated components.
#Though we donât use this further here, it can be used to identify markers that are strongly correlated with cellular heterogeneity, 
#but may not have passed through variable gene selection. The results of the projected PCA can be explored by setting use.full=T in the functions below.
hab = ProjectPCA(hab)

# Examine  and visualize PCA results a few different ways
PrintPCA(hab, pcs.print = 1:5, genes.print = 5, use.full = TRUE)

VizPCA(hab, 1:4)
PCAPlot(hab, 1,2)
PCHeatmap(hab, pc.use = 1:12, cells.use = 500, do.balanced = TRUE, label.columns = FALSE, use.full = FALSE)

# NOTE: This process can take a long time for big datasets, comment out for expediency.
# More approximate techniques such as those implemented in PCElbowPlot() can be used to reduce computation time
PCElbowPlot(hab)


hab = JackStraw(hab, num.replicate = 100, do.print = T) 
JackStrawPlot(hab,PCs=1:30)
#looks like 19 PCs are significant, use 20

#save.SNN=T saves the SNN so that the  SLM algorithm can be rerun using the same graph, but with a different resolution value (see docs for full details)
hab = FindClusters(hab, pc.use = 1:20, resolution = 1.2, print.output = T, save.SNN = T)

#runTSNE
hab = RunTSNE(hab, dims.use = 1:20, do.fast = F)

#plot TSNE
TSNEPlot(hab, do.label = T)

save(hab, file = file.path('/Volumes/neurobio/MICROSCOPE/Daniel/inDrops/data/habenula', 'habClustering_14dim_1.2res.Robj'))

FeaturePlot(hab, c("nUMI"),cols.use = c("grey","blue"))

VlnPlot(hab,c('Snap25','Syn1','Slc32a1', 'Slc17a6'), use.raw = F, y.log = F)
VlnPlot(hab,c('Esr1', 'Gal','Crh','Dbh','Th','Pcp2'), use.raw = F, y.log = F)
VlnPlot(hab,c('Olig1','Pecam1','C1qb','C1qa', 'Aldh1l1','Aldoc'), use.raw = F, y.log = F)
VlnPlot(hab,c('Slc17a7','Slc17a6','Slc17a8','Slc32a1', 'Gad1','Gad2'), use.raw = F, y.log = F)

#find cells that are from Left
isLeft = grepl('L',hab@cell.names, ignore.case=TRUE)
dataL =data.frame(isLeft, row.names = hab@cell.names)

hab = AddMetaData(hab, dataL, "isLeft")


numCells = c()
percentLeft = c()

for( i in 0:max(as.integer(hab@data.info$res.1.2))) {
  numCells[i+1] = (sum(hab@data.info$res.1.2==i))
  percentLeft[i+1] = sum(as.integer(hab@data.info$res.1.2==i)*as.integer(hab@data.info$isLeft))/numCells[i+1]
}

# subset data that looks to be in neuronal clusters (and one olig cluster [4] which has nuerons): 0, 4, 5, 6, 7, 8, 11, 14, 15, 22
hab2 = SubsetData(hab, ident.use = c('0','4','5','6','7','8','11','14','15','22'))

#detect variable geens
# I try to get roughly ~400 genes
hab2 = MeanVarPlot(hab2 ,fxn.x = expMean, fxn.y = logVarDivMean, x.low.cutoff = 0.0125, x.high.cutoff = 3, y.cutoff = 2.25, do.contour = F, num.bin = 100, do.text = F)

hab2 = PCA(hab2, pc.genes = hab2@var.genes, do.print = TRUE, pcs.print = 5, genes.print = 5)

#ProjectPCA scores each gene in the dataset (including genes not included in the PCA) based on their correlation with the calculated components.
#Though we donât use this further here, it can be used to identify markers that are strongly correlated with cellular heterogeneity, 
#but may not have passed through variable gene selection. The results of the projected PCA can be explored by setting use.full=T in the functions below.
hab2 = ProjectPCA(hab2)

# Examine  and visualize PCA results a few different ways
PrintPCA(hab2, pcs.print = 1:5, genes.print = 5, use.full = TRUE)

VizPCA(hab2, 1:4)
PCAPlot(hab2, 1,2)
PCHeatmap(hab2, pc.use = 1:12, cells.use = 500, do.balanced = TRUE, label.columns = FALSE, use.full = FALSE)

# NOTE: This process can take a long time for big datasets, comment out for expediency.
# More approximate techniques such as those implemented in PCElbowPlot() can be used to reduce computation time
PCElbowPlot(hab2)


hab2 = JackStraw(hab2, num.replicate = 100, do.print = T) 
JackStrawPlot(hab2,PCs=1:30)
#looks like 8 PCs are significant, arbitrarily choose 15

#save.SNN=T saves the SNN so that the  SLM algorithm can be rerun using the same graph, but with a different resolution value (see docs for full details)
hab2 = FindClusters(hab2, pc.use = 1:15, resolution = 1.2, print.output = T, save.SNN = T)

#runTSNE
hab2 = RunTSNE(hab2, dims.use = 1:15, do.fast = F)

#plot TSNE
TSNEPlot(hab2, do.label = T)

VlnPlot(hab2,c('Snap25','Syn1','Slc32a1', 'Slc17a6'), use.raw = F, y.log = F)
VlnPlot(hab,c('Esr1', 'Gal','Crh','Dbh','Th','Pcp2'), use.raw = F, y.log = F)
VlnPlot(hab2,c('Olig1','Pecam1','Mbp','C1qa', 'Aldh1l1','Aldoc'), use.raw = F, y.log = F)
VlnPlot(hab2,c('Slc17a7','Slc17a6','Slc17a8','Slc32a1', 'Gad1','Gad2'), use.raw = F, y.log = F)
VlnPlot(hab2,c('Pvalb','Sst','Vip','Htr2c', 'Cck','Calb2'), use.raw = F, y.log = F)

VlnPlot(hab2,c('Chat','Slc18a3','Slc5a7','Slc18a2'), use.raw = F, y.log = F)


numCells2 = c()
percentLeft2 = c()

for( i in 0:max(as.integer(hab2@data.info$res.1.2))) {
  numCells2[i+1] = (sum(hab2@data.info$res.1.2==i))
  percentLeft2[i+1] = sum(as.integer(hab2@data.info$res.1.2==i)*as.integer(hab2@data.info$isLeft))/numCells2[i+1]
}

save(hab2, file = file.path('/Volumes/neurobio/MICROSCOPE/Daniel/inDrops/data/habenula', 'hab2Clustering_15dim_1.2res_neuronEnriched.Robj'))

savePath = '/Volumes/neurobio/MICROSCOPE/Daniel/inDrops/data/habenula'
hab2.markers = FindAllMarkers(hab2, only.pos = T, min.pct = 0.1, thresh.use = 0.25)
write.csv(hab2.markers,file = file.path(savePath, 'hab2_neuronEnriched_unique_geneMarkers.csv'))

VlnPlot(hab2,c('Lmo3','Penk','Syt15','Wif1', 'Spata7','Th'), use.raw = F, y.log = F)
VlnPlot(hab2,c('Gabra1','Gabrb1','Cacng2','Cacna2d1', 'C1ql3','Pcdh10'), use.raw = F, y.log = F)
VlnPlot(hab2,c('Cacna2d1', 'Cacna2d2','Cacna2d3'), use.raw = F, y.log = F)
VlnPlot(hab2,c('Fos', 'Fosb','Jun', 'Egr1','Nr4a2','Junb'), use.raw = F, y.log = F)
VlnPlot(hab2,c('Syn2', 'Slc6a1','Sez6l', 'Kalrn','Gria3'), use.raw = F, y.log = F)

#12/16/16
setwd('/Volumes/neurobio/MICROSCOPE/Daniel/inDrops/data/habenula')
load("hab2Clustering_15dim_1.2res_neuronEnriched.Robj")

#look for medial vs lateral habenula markers
VlnPlot(hab2,c('Tac1', 'Oprm1','Chat', 'Il18','Drd2','Htr2c','Pcdh10','Gpr151', 'Slc17a7','Slc17a6'), use.raw = F, y.log = F)
VlnPlot(hab2,c('Tac1', 'Chat', 'Htr2c','Pcdh10','Gpr151', 'Slc17a7','Slc17a6', 'Oprm1', 'Drd2'), use.raw = F, y.log = F)
